In [5]:
import os
import torch
import shutil
import zipfile
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import multiprocessing
import torch.optim as optim
import torch.utils.data as data
import time, os, copy, argparse
from torchsummary import summary
from torch.optim import lr_scheduler
from matplotlib import pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from torchvision import datasets, models, transforms

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def extract_zip_file(zip_path, dest_path):
    """
    This function extracts a zip file to a specified destination path.

    Parameters:
    zip_path (str): The path to the zip file that needs to be extracted.
    dest_path (str): The path where the zip file should be extracted to.

    Returns:
    None
    """
    # Open the zip file in read mode
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all the contents of the zip file to dest_path
        zip_ref.extractall(dest_path)

# Usage
# The function is called with the path of the zip file and the destination path as arguments
extract_zip_file('/content/drive/MyDrive/data.zip', 'data')

In [3]:
# Load the training data
X_train = pd.read_csv('data/X_train.csv')  # Load features from CSV file
Y_train = pd.read_csv('data/Y_train.csv')  # Load labels from CSV file

# Add the output column to X_train
X_train['output'] = Y_train['output']  # Add 'output' column to X_train dataframe

# Print the count of unique values in the 'output' column. This is useful to understand the distribution of values.
print(X_train['output'].value_counts())

# Print the shape of the dataframes to understand the number of rows and columns
print("Shape of X_train: ", X_train.shape)
print("Shape of Y_train: ", Y_train.shape)

output
0    532
1    507
Name: count, dtype: int64
(1039, 16)
(1039, 1)


In [4]:
# Split the data into training and validation sets
# The stratify parameter makes a split so that the proportion of values in the sample produced will be the same as the proportion of values provided to parameter stratify.
# For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.
train, valid = train_test_split(X_train, test_size=0.10, stratify=X_train['output'], random_state=42)

def process_df(df, folder):
    """
    This function processes a dataframe and copies images to a specified folder.

    Parameters:
    df (DataFrame): The dataframe to process.
    folder (str): The folder to copy images to.

    Returns:
    None
    """
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Get the output value (0 or 1)
        output = row['output']

        # Create the output directory if it doesn't exist
        output_dir = os.path.join('IMAGES_', folder, str(output))
        os.makedirs(output_dir, exist_ok=True)

        # Copy the image to the output directory
        shutil.copy(row['image_path'], output_dir)

# Process the training and validation dataframes
process_df(train, 'train')
process_df(valid, 'valid')

In [7]:
train_directory = 'IMAGES_/train'
valid_directory = 'IMAGES_/valid'

In [23]:
# Mode for training ('finetune' indicates that a pre-trained model will be fine-tuned)
train_mode = 'finetune'

# Path to save the fine-tuned model
PATH = "model_finetune.pth"

# Batch size for training
bs = 8

# Number of epochs for training
num_epochs = 10

# Number of classes in the dataset
num_classes = 2

# Number of workers for data loading
num_cpu = multiprocessing.cpu_count()

# Define the transformations to be applied to the images
image_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),  # Randomly resize and crop the image
        transforms.RandomRotation(degrees=15),  # Randomly rotate the image
        transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
        transforms.CenterCrop(size=224),  # Crop the image from the center
        transforms.ToTensor(),  # Convert the image to a PyTorch tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),  # Resize the image
        transforms.CenterCrop(size=224),  # Crop the image from the center
        transforms.ToTensor(),  # Convert the image to a PyTorch tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image
    ])
}

# Load the datasets from the specified folders and apply the transformations
dataset = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),  # Training dataset
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])  # Validation dataset
}


In [24]:
# Calculate the size of the training and validation datasets
dataset_sizes = {
    'train': len(dataset['train']),  # Size of the training dataset
    'valid': len(dataset['valid'])  # Size of the validation dataset
}

# Create DataLoader objects for the training and validation datasets
# These will provide batches of data to the training loop
# They also handle shuffling and parallel data loading
dataloaders = {
    'train': data.DataLoader(dataset['train'], batch_size=bs, shuffle=True,
                             num_workers=num_cpu, pin_memory=True, drop_last=True),  # DataLoader for the training data
    'valid': data.DataLoader(dataset['valid'], batch_size=bs, shuffle=True,
                             num_workers=num_cpu, pin_memory=True, drop_last=True)  # DataLoader for the validation data
}

# Get the class names or target labels from the training dataset
class_names = dataset['train'].classes

# Print the class names
print("Classes:", class_names)

Classes: ['0', '1']


In [25]:
# Print the sizes of the training and validation datasets
print("Training-set size:", dataset_sizes['train'])
print("Validation-set size:", dataset_sizes['valid'])

# Set the default device to GPU if it's available, otherwise use CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Check the training mode
if train_mode == 'finetune':
    # Load a pre-trained ResNet18 model
    print("\nLoading ResNet18 for fine-tuning...\n")
    model_ft = models.resnet18(pretrained=True)

    # Get the number of features in the last layer (fc layer)
    num_ftrs = model_ft.fc.in_features

    # Modify the last layer to match the number of classes in the dataset
    model_ft.fc = nn.Linear(num_ftrs, num_classes)

elif train_mode == 'scratch':
    # Load a custom VGG11 model for training from scratch
    print("\nLoading VGG11 for training from scratch...\n")
    model_ft = MyVGG11(in_ch=3, num_classes=2)

    # Set the number of epochs to a higher value for training from scratch
    num_epochs = 100

elif train_mode == 'transfer':
    # Load a pre-trained MobileNetV2 model for transfer learning
    print("\nLoading MobileNetV2 as feature extractor...\n")
    model_ft = models.mobilenet_v2(pretrained=True)

    # Freeze all the layers except the last convolution block and fully connected (fc) layers
    for params in list(model_ft.parameters())[0:-5]:
        params.requires_grad = False

    # Get the number of features in the last layer of the classifier
    num_ftrs = model_ft.classifier[-1].in_features

    # Modify the classifier to match the number of classes in the dataset
    model_ft.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=False),
        nn.Linear(in_features=num_ftrs, out_features=num_classes, bias=True)
    )

Training-set size: 935 
Validation-set size: 104

Loading resnet18 for finetuning ...



In [26]:
# Transfer the model to GPU if available
model_ft = model_ft.to(device)

# Print the summary of the model
print('Model Summary:-\n')

# Enumerate through the model parameters and print them
for num, (name, param) in enumerate(model_ft.named_parameters()):
    print(num, name, param.requires_grad)  # Print the parameter number, name, and whether it requires gradient

# Use the summary function from the torchsummary package to print a detailed summary of the model
summary(model_ft, input_size=(3, 224, 224))

# Print the model architecture
print(model_ft)

Model Summary:-

0 conv1.weight True
1 bn1.weight True
2 bn1.bias True
3 layer1.0.conv1.weight True
4 layer1.0.bn1.weight True
5 layer1.0.bn1.bias True
6 layer1.0.conv2.weight True
7 layer1.0.bn2.weight True
8 layer1.0.bn2.bias True
9 layer1.1.conv1.weight True
10 layer1.1.bn1.weight True
11 layer1.1.bn1.bias True
12 layer1.1.conv2.weight True
13 layer1.1.bn2.weight True
14 layer1.1.bn2.bias True
15 layer2.0.conv1.weight True
16 layer2.0.bn1.weight True
17 layer2.0.bn1.bias True
18 layer2.0.conv2.weight True
19 layer2.0.bn2.weight True
20 layer2.0.bn2.bias True
21 layer2.0.downsample.0.weight True
22 layer2.0.downsample.1.weight True
23 layer2.0.downsample.1.bias True
24 layer2.1.conv1.weight True
25 layer2.1.bn1.weight True
26 layer2.1.bn1.bias True
27 layer2.1.conv2.weight True
28 layer2.1.bn2.weight True
29 layer2.1.bn2.bias True
30 layer3.0.conv1.weight True
31 layer3.0.bn1.weight True
32 layer3.0.bn1.bias True
33 layer3.0.conv2.weight True
34 layer3.0.bn2.weight True
35 layer3.0.b

In [27]:
# Define the loss function
# CrossEntropyLoss is used for multi-class classification tasks
criterion = nn.CrossEntropyLoss()

# Define the optimizer
# Stochastic Gradient Descent (SGD) is used as the optimizer
# The learning rate is set to 0.001 and the momentum is set to 0.9
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Define the learning rate scheduler
# The learning rate will be reduced by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [28]:
print("\nTraining:-\n")

def train_model(model, criterion, optimizer, scheduler, num_epochs=30):
    """
    This function trains the model for a specified number of epochs and optimizes it.

    Parameters:
    model (torch.nn.Module): The model to train.
    criterion (torch.nn.modules.loss): The loss function.
    optimizer (torch.optim): The optimizer.
    scheduler (torch.optim.lr_scheduler): The learning rate scheduler.
    num_epochs (int): The number of epochs to train the model.

    Returns:
    model (torch.nn.Module): The trained model.
    """
    # Record the start time
    since = time.time()

    # Initialize the best model weights and accuracy
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Initialize the TensorBoard writer
    writer = SummaryWriter()

    # Loop over the number of epochs
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set the model to training mode
            else:
                model.eval()   # Set the model to evaluation mode

            # Initialize the running loss and correct predictions
            running_loss = 0.0
            running_corrects = 0

            # Iterate over the data
            for inputs, labels in dataloaders[phase]:
                # Transfer the inputs and labels to the device
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Update the running loss and correct predictions
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()

            # Step the scheduler if in training phase
            if phase == 'train':
                scheduler.step()

            # Calculate the epoch loss and accuracy
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = torch.tensor(running_corrects).double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Record the training loss and accuracy for each phase in TensorBoard
            if phase == 'train':
                writer.add_scalar('Train/Loss', epoch_loss, epoch)
                writer.add_scalar('Train/Accuracy', epoch_acc, epoch)
                writer.flush()
            else:
                writer.add_scalar('Valid/Loss', epoch_loss, epoch)
                writer.add_scalar('Valid/Accuracy', epoch_acc, epoch)
                writer.flush()

            # Deep copy the model if the current accuracy is the best
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    # Calculate and print the time taken for training
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Load the best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=num_epochs)

# Save the entire model
print("\nSaving the model...")



Training:-

Epoch 0/9
----------
train Loss: 0.4994 Acc: 0.7551
valid Loss: 0.3439 Acc: 0.8269

Epoch 1/9
----------
train Loss: 0.3971 Acc: 0.8289
valid Loss: 0.3229 Acc: 0.8654

Epoch 2/9
----------
train Loss: 0.3961 Acc: 0.8225
valid Loss: 0.5342 Acc: 0.8077

Epoch 3/9
----------
train Loss: 0.2992 Acc: 0.8684
valid Loss: 0.4526 Acc: 0.8269

Epoch 4/9
----------
train Loss: 0.2699 Acc: 0.8888
valid Loss: 0.4181 Acc: 0.8750

Epoch 5/9
----------
train Loss: 0.2484 Acc: 0.8984
valid Loss: 0.4374 Acc: 0.8654

Epoch 6/9
----------
train Loss: 0.2434 Acc: 0.8952
valid Loss: 0.5149 Acc: 0.8558

Epoch 7/9
----------
train Loss: 0.2572 Acc: 0.9005
valid Loss: 0.4959 Acc: 0.8462

Epoch 8/9
----------
train Loss: 0.1306 Acc: 0.9433
valid Loss: 0.4621 Acc: 0.8365

Epoch 9/9
----------
train Loss: 0.1162 Acc: 0.9444
valid Loss: 0.4603 Acc: 0.8558

Training complete in 7m 44s
Best val Acc: 0.875000

Saving the model...


In [29]:
# Save the trained model to a file
# 'model_ft' is the trained model and 'PATH' is the location where the model will be saved
torch.save(model_ft, PATH)

In [30]:
PATH="/content/drive/MyDrive/model_finetune.pth"
torch.save(model_ft, PATH)